In [11]:
import os
import sys
#sys.path.append("/workspace")
import numpy as np
import tensorflow as tf
from src.data.general_processor import Utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
from sklearn.preprocessing import minmax_scale
import pathlib
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


from src.models.model_architectures.model_1DCNN import HopefullNet
from src.data.make_dataset import BCIDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
root_path = pathlib.Path().resolve().parents[0]

# DATA
# data_root = "C:/Users/annag/OneDrive - Danmarks Tekniske Universitet/Semester_04/Special_Course_BCI/03_code/BCI_stroke_rehab/data/raw/"
data_root = "C:/Users/annag/OneDrive/Dokumente/Special_Course/03_code/data/raw/"

# MODEL
save_path = os.path.join(root_path, "models/1D_CNN/ours_6_pairs/fine_tuned/physionet_base/")

## Device settings
tf.autograph.set_verbosity(0)
physical_devices = tf.config.experimental.list_physical_devices('CPU')

#Params    
channels = [["C3","C4"],["F3","F4"],["P3","P4"],["FP1","FP2"],["F7","F8"],["T3","T4"]]
two_class = True

if two_class:
    input_shape = (None, 500, 2)
    loss = tf.keras.losses.binary_crossentropy
else:
    input_shape = (None, 640, 2)
    loss = tf.keras.losses.categorical_crossentropy

In [15]:
# Test subjects for testing transfer learning
test_subjects = ["dani", "ivo", "pablo", "huiyu", "manu", "fabio", "anna", "luisa", "sarah", "irene", "jan"]
#test_subjects = ["fabio"]

measurements = []

# Get dataset
trainingset = BCIDataset(data_root, test_subjects, [], measurement_length=4)
trainingset.validate_data()
trainingset.apply_bandpass_filtering(selected_data="sample")

Subjects: ['dani', 'ivo', 'pablo', 'huiyu', 'manu', 'fabio', 'anna', 'luisa', 'sarah', 'irene', 'jan']
Total invalid samples: 60


In [16]:
save_path

'C:\\Users\\annag\\OneDrive\\Dokumente\\Special_Course\\03_code\\models/1D_CNN/ours_6_pairs/fine_tuned/physionet_base/'

In [23]:
log_accuracies = {}
for subj in test_subjects: 
    # Modelname
    modelname = "finetuned_sub_"+subj+"_2023-05-05_11-32-36.h5"

    log_accuracies[subj] = {}
    # Load data
    x, y, ch_pairs = trainingset.load_subject_data(subj, channels)
    indexes = range(0,len(x))

    # Reshape for scaling
    reshaped_x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    
#     train_idx, test_idx, y_train_raw, y_test = train_test_split(indexes,
#                                                                 y,
#                                                                 stratify=y,
#                                                                 test_size=0.20,
#                                                                 random_state=42)
    # Get train-/test- indices of 5 fold Cross Validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for i, (train_index, test_index) in enumerate(skf.split(reshaped_x, y)):
        # Get model path
        model_path = os.path.join(save_path+"sub_"+subj+"/"+"fold_"+str(i)+"/", modelname)
        
#         test_index = [1,7,10,14,16,23,26,28,34,46,52,55,60,62,65,67,69,75,81,89,91,  93,  94, 103, 111, 113, 118, 120, 121,
#                       129, 147, 148, 154, 156, 158, 159, 162, 177, 183, 189, 191 ,193 ,194 ,196, 207, 218, 226 ,231 ,241,
#                       242, 248 ,252, 255, 259, 260 ,261 ,265, 271, 281, 289, 290 ,304, 306, 307 ,310 ,314 ,315, 325, 339,
#                       342, 343 ,354, 356, 358 ,367 ,369, 372, 379, 387 ,391, 411, 419, 429, 434, 442, 448 ,449, 450, 451,
#                       458, 461, 468, 470,472 ,473, 481, 483 ,484, 486, 494, 495, 504 ,505, 517, 521, 526 ,527, 528, 532,
#                       542 ,545 ,547 ,553, 554, 557 ,560, 562, 581 ,593, 596 ,605, 606, 610, 611, 617, 624, 630, 648, 649,
#                       650, 658, 661, 662, 663, 664, 669, 670, 679, 689, 697, 707, 709, 715, 719]
    
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

        # x_train_raw = reshaped_x[train_index]
        x_test_raw = reshaped_x[test_index]
        # y_train_raw = y[train_index]
        y_test = y[test_index]
        ch_pairs_test = ch_pairs[test_index]
        
        # Normalize array
        reshaped_x_scaled = minmax_scale(x_test_raw, axis=1)
        # x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], int(reshaped_x_scaled.shape[1]/2),2).astype(np.float64)
        x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], x.shape[1], x.shape[2]).astype(np.float64)
        x_test = np.swapaxes(x_test,1,2)
    
        # Transform labels in int-values (0/1)
        y_test_01 = []
        for y_label in y_test:
            if y_label == 'L':
                y_test_01.append(0)
            elif y_label == 'R':
                y_test_01.append(1)
            else:
                print("Test Labels are different than L or R...")

        y_test = np.array(y_test_01)

        ## Load model   
        model = HopefullNet(inp_shape = (input_shape[1],input_shape[2]), two_class=two_class)
        model.build(input_shape)
        model.load_weights(model_path)

        learning_rate = 1e-4
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
        
        print(np.shape(x_test))
        testLoss, testAcc = model.evaluate(x_test, y_test)
        print('\nAccuracy:', testAcc)
        print('\nLoss: ', testLoss)

        ## Test model with subject data 
        for ch_pair in channels:
            ch_pair_name = ' '.join(ch_pair)
            print(f"Test model with subject {subj}: and channel pair: {ch_pair}\n")
            
            try:
                len(log_accuracies[subj][ch_pair_name])
            except:
                log_accuracies[subj][ch_pair_name] = []
            

            # Get samples of channel pair
            idx = np.where(ch_pairs_test == ch_pair)[0]
            x_test_ch = x_test[idx]
            y_test_ch = y_test[idx]

            yPred = model.predict(x_test_ch)

            # convert from one hot encode in string
            yTestClass = y_test_ch
            yPredClass = []
            for label in yPred:
                if label<0.5:
                    yPredClass.append(0)
                elif label>=0.5:
                    yPredClass.append(1)
                else:
                    print("Label not found.")
            yPredClass = np.array(yPredClass)
            
            print(np.shape(yPredClass))
            print(np.shape(yTestClass))

            target_names = ["L", "R"]
                
            # Calc accuracy
            testAcc_ch = accuracy_score(yTestClass, yPredClass)
            print(f"Acc={testAcc_ch}")
            log_accuracies[subj][ch_pair_name].append(testAcc_ch)
            
            

#             print('\n Classification report \n\n',
#               classification_report(
#                   yTestClass,
#                   yPredClass,
#                    target_names=target_names
#               )
#             )

#             print('\n Confusion matrix \n\n',
#               confusion_matrix(
#                   yTestClass,
#                   yPredClass,
#               )
#             )

Get data from subjects: ['dani']
Fold 0:
  Train: index=[  0   2   3   5   8   9  10  11  12  13  14  15  16  17  18  19  20  21
  22  24  25  26  27  28  29  30  31  34  37  38  39  40  41  42  44  45
  46  47  49  50  52  53  54  57  58  59  60  61  62  63  64  65  67  68
  70  72  73  74  75  77  78  79  80  82  83  84  85  89  90  91  92  93
  94  95  97  98  99 100 101 102 103 105 106 107 108 109 110 111 113 114
 115 117 119 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 140 141 142 143 144 146 147 148 149 150 151 152 153 157 159 160 162
 163 164 165 166 167 168 169 171 172 173 174 175 177 179 180 181 182 183
 184 185 186 187 188 189 191 192 193 196 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 213 214 215 217 218 219 220 221 222 224 225 226
 227 229 230 231 232 233 234 235 236 237 238 239 240 243 244 245 246 247
 248 250 251 253 254 256 257 258 259 260 261 262 263 265 267 269 272 273
 274 275 276 277 278 279 280 281 282 283 284 285 287 288 289 290 291

KeyboardInterrupt: 

In [19]:
len(test_index)

144

In [26]:
# Calc mean accuracies
for sub in test_subjects:
    print(f"Subject {sub}:\n")
    for ch_pair in channels:
        ch_pair_name = ' '.join(ch_pair)
        mean_acc = np.mean(log_accuracies[sub][ch_pair_name])
        print(f"Mean acc {ch_pair_name} = {mean_acc}")
    print("\n")

Subject dani:

Mean acc C3 C4 = 0.99
Mean acc F3 F4 = 0.9563892339544513
Mean acc P3 P4 = 0.9286153846153846
Mean acc FP1 FP2 = 0.993103448275862
Mean acc F7 F8 = 0.9894736842105264
Mean acc T3 T4 = 0.9843076923076923


Subject ivo:

Mean acc C3 C4 = 0.99
Mean acc F3 F4 = 0.9388944099378882
Mean acc P3 P4 = 0.9160012740882305
Mean acc FP1 FP2 = 0.980952380952381
Mean acc F7 F8 = 0.992
Mean acc T3 T4 = 0.9846153846153847


Subject pablo:

Mean acc C3 C4 = 1.0
Mean acc F3 F4 = 0.9488215488215488
Mean acc P3 P4 = 0.9764895330112722
Mean acc FP1 FP2 = 1.0
Mean acc F7 F8 = 1.0
Mean acc T3 T4 = 1.0


Subject huiyu:

Mean acc C3 C4 = 1.0
Mean acc F3 F4 = 0.913913043478261
Mean acc P3 P4 = 0.9733706246292059
Mean acc FP1 FP2 = 1.0
Mean acc F7 F8 = 1.0
Mean acc T3 T4 = 0.9913043478260869


Subject manu:

Mean acc C3 C4 = 1.0
Mean acc F3 F4 = 0.8916452293836195
Mean acc P3 P4 = 0.982
Mean acc FP1 FP2 = 1.0
Mean acc F7 F8 = 1.0
Mean acc T3 T4 = 0.990909090909091


Subject fabio:

Mean acc C3 C4 =